In [1]:
import numpy as np
import pandas as pd
import itertools
from zipfile import ZipFile
from sklearn.model_selection import train_test_split

In [47]:
# Read data from CSV file.
with ZipFile('Murder_Data.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

df = pd.read_csv("SHR76_20.csv")
df.info()

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 827219 entries, 0 to 827218
Data columns (total 31 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   ID            827219 non-null  object 
 1   CNTYFIPS      827219 non-null  object 
 2   Ori           827219 non-null  object 
 3   State         827219 non-null  object 
 4   Agency        827219 non-null  object 
 5   Agentype      827219 non-null  object 
 6   Source        827219 non-null  object 
 7   Solved        827219 non-null  object 
 8   Year          827219 non-null  int64  
 9   StateName     22468 non-null   object 
 10  Month         827219 non-null  object 
 11  Incident      827219 non-null  int64  
 12  ActionType    827219 non-null  object 
 13  Homicide      827219 non-null  object 
 14  Situation     827219 non-null  object 
 15  VicAge        827219 non-null  int64  
 16  VicSex        827219 non-null  object 
 17  VicRace       827219 non-null  object 
 18  VicE

In [56]:
df[df['CNTYFIPS'].str.contains("Kanabec")]

,ID,CNTYFIPS,Ori,State,Agency,Agentype,Source,Solved,Year,StateName,...,OffRace,OffEthnic,Weapon,Relationship,Circumstance,Subcircum,VicCount,OffCount,FileDate,MSA
436204,197808001MN03300,"Kanabec, MN",MN03300,Minnesota,Kanabec County,Sheriff,FBI,Yes,1978,NaN,...,White,Unknown or not reported,Shotgun,Wife,Other,NaN,0,0,30180.0,Rural Minnesota
436205,198402001MN03300,"Kanabec, MN",MN03300,Minnesota,Kanabec County,Sheriff,FBI,Yes,1984,NaN,...,White,Unknown or not reported,Shotgun,Wife,Other,NaN,0,0,62684.0,Rural Minnesota
436206,199406001MN03300,"Kanabec, MN",MN03300,Minnesota,Kanabec County,Sheriff,FBI,Yes,1994,NaN,...,White,Unknown or not reported,Rifle,Father,Other,NaN,0,0,11195.0,Rural Minnesota
436207,199604001MN03300,"Kanabec, MN",MN03300,Minnesota,Kanabec County,Sheriff,FBI,No,1996,NaN,...,Unknown,Unknown or not reported,"Handgun - pistol, revolver, etc",Relationship not determined,Circumstances undetermined,NaN,0,0,101797.0,Rural Minnesota
436208,199606001MN03300,"Kanabec, MN",MN03300,Minnesota,Kanabec County,Sheriff,FBI,Yes,1996,NaN,...,White,Unknown or not reported,"Handgun - pistol, revolver, etc",Wife,Other,NaN,0,0,101797.0,Rural Minnesota
436209,199607001MN03300,"Kanabec, MN",MN03300,Minnesota,Kanabec County,Sheriff,FBI,Yes,1996,NaN,...,White,Unknown or not reported,"Blunt object - hammer, club, etc",Wife,Other arguments,NaN,0,0,101797.0,Rural Minnesota
436210,199611001MN03300,"Kanabec, MN",MN03300,Minnesota,Kanabec County,Sheriff,FBI,Yes,1996,NaN,...,White,Unknown or not reported,"Firearm, type not stated",Acquaintance,Other arguments,NaN,0,0,101797.0,Rural Minnesota
436211,200103001MN03300,"Kanabec, MN",MN03300,Minnesota,Kanabec County,Sheriff,FBI,Yes,2001,NaN,...,White,Unknown or not reported,"Handgun - pistol, revolver, etc",In-law,Other,NaN,0,0,42002.0,Rural Minnesota
436212,200308001MN03300,"Kanabec, MN",MN03300,Minnesota,Kanabec County,Sheriff,FBI,Yes,2003,NaN,...,White,Unknown or not reported,Rifle,Mother,Argument over money or property,NaN,0,0,30404.0,Rural Minnesota
436213,200408001MN03300,"Kanabec, MN",MN03300,Minnesota,Kanabec County,Sheriff,FBI,Yes,2004,NaN,...,White,Unknown or not reported,Knife or cutting instrument,Acquaintance,Circumstances undetermined,NaN,0,0,82504.0,Rural Minnesota


## Cleaning Data

In [69]:
def clean_dataframe(df):
    original_data = df.copy(deep=True)
    solved, unsolved = split_solved(original_data)
    data = remove_col(solved, ['Situation', 'Incident', 'Ori', 'StateName'])
    data = split_filedate(data)
    data = split_county_area(data)
    data = fill_unknown(data)
    data = delete_val(data, ['OffSex', 'OffRace', 'VicSex', 'VicRace'], ['Unknown', 'Unknown', 'Unknown', 'Unknown'])
    data = clean_unk(data)
#     data = del_agentype(data, 'Agentype')
    return data

# Split the dataframe into a solved and unsolved dataset
def split_solved(df):
    grouped = df.groupby(df.Solved)
    return grouped.get_group("Yes"), grouped.get_group("No")

# Fill in missing data (NaN) with Unknown value
def fill_unknown(df):
    return df.fillna("Unknown")

# Remove column(s)
def remove_col(df, cols):
    df = df.drop(cols, axis=1)
    return df

# Delete rows with specific column value
def delete_val(df, cols, values):
    for i in range(len(cols)):
        df = df.drop(df[df[cols[i]] == values[i]].index)
    return df

# Everything that is unknown/undetermined/not specified/not reported/not determined, change to Unknown
def clean_unk(df):
    df.loc[df['OffAge'] == 999, 'OffAge'] = 'Unknown'
    df.loc[df['VicAge'] == 999, 'VicAge'] = 'Unknown'
    for col in df.columns:
        try:
            df.loc[df[col].str.contains("unknown|undetermined|not specified|not reported|not determined|^na\Z|Not enough information to determine|^$", regex = True), col] = 'Unknown'
        except:
            continue
    return df

# Delete Agency Type which is equal to 4 (unclear)
def del_agentype(df, col):
    return df.drop(df[df[col] == '4'].index)

# Split the FileDate column into three columns
def split_filedate(df):
    df['FileDate'] = df['FileDate'].astype(str)
    df.insert(0, 'FileYear', df['FileDate'].map(lambda x : x[len(x)-4:len(x)-2]), True)
    df.insert(0, 'FileDay', df['FileDate'].map(lambda x : x[len(x)-6:len(x)-4]), True)
    df.insert(0, 'FileMonth', df['FileDate'].map(lambda x : x[0:len(x)-6]), True)
    df['FileYear'] = df['FileYear'].astype(str)
    df['FileMonth'] = df['FileMonth'].astype(str)
    df['FileDay'] = df['FileDay'].astype(str)
    df = df.drop(['FileDate'], axis=1)
    return df

# Split CNTYFIPS and MSA columns into County and Area
def split_county_area(df):
    df.insert(0, 'County', df['CNTYFIPS'].map(lambda x : str(x).split(',')[0]), True)
    df.insert(0, 'Area',df['MSA'].map(lambda x :  str(x).split(',')[0] ) ,True)
    df = df.drop(['CNTYFIPS'], axis=1)
    df = df.drop(['MSA'], axis=1)
    return df

# Split dataframe into small train and test set, stratified on cols
def split_stratify(df, cols, train_frac, test_frac):
    # Get unique combinations of columns
    unique_vals = [np.unique(df[[col]].values) for col in cols]
    combinations = list(itertools.product(*unique_vals))
    
    # Create df for each combination and sample non-random from df
    train, test = [], []
    for combi in combinations:
        binned_df = df.loc[(df[cols[0]] == combi[0]) & (df[cols[1]] == combi[1]) & (df[cols[2]] == combi[2]) & (df[cols[3]] == combi[3])]
        train.append(binned_df.sample(frac=train_frac, replace=True, random_state=1))
        test.append(binned_df.sample(frac=test_frac, replace=True, random_state=1))
        
    # Return training df and test df
    return pd.concat(train, ignore_index=True), pd.concat(test, ignore_index=True)      

In [70]:
cleaned = clean_dataframe(df)

In [72]:
cleaned[cleaned['County'] == 'Kanabec']

,Area,County,FileMonth,FileDay,FileYear,ID,State,Agency,Agentype,Source,...,OffAge,OffSex,OffRace,OffEthnic,Weapon,Relationship,Circumstance,Subcircum,VicCount,OffCount
436204,Rural Minnesota,Kanabec,3,01,80,197808001MN03300,Minnesota,Kanabec County,Sheriff,FBI,...,66,Male,White,Unknown,Shotgun,Wife,Other,Unknown,0,0
436205,Rural Minnesota,Kanabec,6,26,84,198402001MN03300,Minnesota,Kanabec County,Sheriff,FBI,...,35,Male,White,Unknown,Shotgun,Wife,Other,Unknown,0,0
436206,Rural Minnesota,Kanabec,1,11,95,199406001MN03300,Minnesota,Kanabec County,Sheriff,FBI,...,17,Male,White,Unknown,Rifle,Father,Other,Unknown,0,0
436208,Rural Minnesota,Kanabec,10,17,97,199606001MN03300,Minnesota,Kanabec County,Sheriff,FBI,...,40,Male,White,Unknown,"Handgun - pistol, revolver, etc",Wife,Other,Unknown,0,0
436209,Rural Minnesota,Kanabec,10,17,97,199607001MN03300,Minnesota,Kanabec County,Sheriff,FBI,...,35,Male,White,Unknown,"Blunt object - hammer, club, etc",Wife,Other arguments,Unknown,0,0
436210,Rural Minnesota,Kanabec,10,17,97,199611001MN03300,Minnesota,Kanabec County,Sheriff,FBI,...,44,Male,White,Unknown,"Firearm, type not stated",Acquaintance,Other arguments,Unknown,0,0
436211,Rural Minnesota,Kanabec,4,20,02,200103001MN03300,Minnesota,Kanabec County,Sheriff,FBI,...,33,Male,White,Unknown,"Handgun - pistol, revolver, etc",In-law,Other,Unknown,0,0
436212,Rural Minnesota,Kanabec,3,04,04,200308001MN03300,Minnesota,Kanabec County,Sheriff,FBI,...,26,Male,White,Unknown,Rifle,Mother,Argument over money or property,Unknown,0,0
436213,Rural Minnesota,Kanabec,8,25,04,200408001MN03300,Minnesota,Kanabec County,Sheriff,FBI,...,21,Male,White,Unknown,Knife or cutting instrument,Acquaintance,Unknown,Unknown,0,0
436214,Rural Minnesota,Kanabec,7,30,09,200902001MN03300,Minnesota,Kanabec County,Sheriff,FBI,...,27,Male,White,Unknown,Knife or cutting instrument,Acquaintance,Robbery,Unknown,1,1
